In [ ]:
import tensorflow as tf
import numpy as np
AUTO = tf.data.experimental.AUTOTUNE
GCS_OUTPUT = 'gs://cloudfire_lyrical-edition-273206/fire_dataset/tfrecords-dataset-eval/'  # prefix for output file names

TARGET_SIZE = [100, 100]


def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar

        # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
        "label": tf.io.FixedLenFeature([], tf.string),  # one bytestring
        "size": tf.io.FixedLenFeature([2], tf.int64),  # two integers
        "one_hot_class": tf.io.VarLenFeature(tf.float32)  # a certain number of floats
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)

    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding

    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.reshape(image, [*TARGET_SIZE, 3])

    class_num = example['class']

    label = example['label']
    height = example['size'][0]
    width = example['size'][1]
    one_hot_class = tf.sparse.to_dense(example['one_hot_class'])
    return image, class_num, label, height, width, one_hot_class


# read from TFRecords. For optimal performance, read from multiple
# TFRecord files at once and set the option experimental_deterministic = False
# to allow order-altering optimizations.

option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

filenames = tf.io.gfile.glob(GCS_OUTPUT + "*.tfrec")
dataset4 = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
dataset4 = dataset4.with_options(option_no_order)
dataset4 = dataset4.map(read_tfrecord, num_parallel_calls=AUTO)

In [ ]:
dataset4

In [ ]:
for image, class_num, label, height, width, one_hot_class in dataset4.take(25):
    print("Image shape {}, {}x{} px, class={} ({:>10}, {})".format(image.numpy().shape, width, height, class_num,label.numpy().decode('utf8'),one_hot_class))

In [ ]:
from matplotlib import pyplot as plt
CLASSES = ['Fire', 'Normal']

def dataset_to_numpy_util(dataset, N):
    dataset = dataset.batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels
'''
def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis=-1)  # one-hot to class number
    correct_label = np.argmax(correct_label, axis=-1) # one-hot to class number
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', shoud be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct
'''
def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1
  
def display_9_images_from_dataset(dataset):
    subplot=331
    plt.figure(figsize=(13,13))
    images, labels = dataset_to_numpy_util(dataset, 9)
    for i, image in enumerate(images):
        title = CLASSES[np.argmax(labels[i], axis=-1)]
        subplot = display_one_flower(image, title, subplot)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()  

    '''
def display_9_images_with_predictions(images, predictions, labels):
    subplot=331
    plt.figure(figsize=(13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower(image, title, subplot, not correct)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    '''


In [ ]:
display_9_images_from_dataset(dataset4)

In [ ]:
from matplotlib import pyplot as plt
CLASSES = ['Fire', 'Normal']

def show_batch(numpy_images, numpy_labels):
    plt.figure(figsize=(10,10))
    for n in range(25):
        plt.subplot(3, 3, 8)
        plt.imshow(numpy_images[n])
        plt.title(CLASSES[numpy_labels[n]==1][0].title())
        plt.axis('off')        

In [ ]:
def dataset_to_numpy_util(dataset4,N):
    dataset = dataset4.batch(N)
    for image, _, label, _, _, _ in dataset:
        numpy_images = image.numpy()
        numpy_labels = label.numpy() 
        show_batch(numpy_images, numpy_labels)      

In [ ]:
dataset_to_numpy_util(dataset4, 25) 

In [ ]:
%%bash
pip install kaggle

In [ ]:
%%bash
gsutil -m cp -r /home/jupyter/FYP_FIRENET/fire-eye gs://cloudfire_lyrical-edition-273206/trained

### Local Train

In [ ]:
%%bash
MODEL_DIR=/home/jupyter/FYP_FIRENET/fire-eye
#rm -rf Trainer.tar.gz fire-eye
gcloud ai-platform local train \
    --module-name=Trainer.config \
    --package-path=${PWD}/Trainer \
    -- \
    --output_dir=$MODEL_DIR\
    --train_steps=50 \
    --learning_rate=0.001 \
    --batch_size=20 \
    --train_data_path=gs://cloudfire_lyrical-edition-273206/fire_dataset/tfrecords-dataset-7/ \
    --eval_data_path=gs://cloudfire_lyrical-edition-273206/fire_dataset/tfrecords-dataset-eval/

### Misc

In [ ]:
%%bash
mkdir /home/jupyter/.kaggle/
mv /home/jupyter/kaggle.json /home/jupyter/.kaggle/

In [ ]:
%%bash
kaggle datasets download --unzip -d phylake1337/fire-dataset

In [ ]:
%%bash
mv /home/jupyter/FYP_FIRENET/fire_dataset /home/jupyter/

In [ ]:
%%bash
gsutil -m mv -r /home/jupyter/fire_images/*.jpg gs://cloudfire_lyrical-edition-273206/fire_dataset/block3/

In [ ]:
%%bash
gsutil -m mv -r /home/jupyter/non_fire_images/*.jpg gs://cloudfire_lyrical-edition-273206/fire_dataset/Normal/

In [ ]:
%%bash
gsutil -m cp -r /home/jupyter/FYP_FIRENET/fire-eye gs://cloudfire_lyrical-edition-273206/model_dir01

In [ ]:
%%bash
cd home/jupyter/FYP_FIRENET
python preprocess.py

### AI platform Train

- setup env variables

In [3]:
import os
PROJECT = "citric-sol-273815" # REPLACE WITH YOUR PROJECT ID
BUCKET = "cloudfire_lyrical-edition-273206" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [4]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


- This allows ai platform to read/write to the staging bucket

In [ ]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud AI Platform API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud AI Platform account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET   

- submits training job to the cloud

In [11]:
%%bash
OUTDIR=gs://cloudfire_lyrical-edition-273206/model_dir77
REGION="us-central1"
JOBNAME=FireSage_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=Trainer.config \
    --package-path=${PWD}/Trainer \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://cloudfire_lyrical-edition-273206 \
    --scale-tier=BASIC_GPU \
    --runtime-version=2.1 \
    --python-version=3.7 \
    -- \
    --output_dir=$OUTDIR \
    --train_steps=500 \
    --learning_rate=0.001 \
    --batch_size=40 \
    --train_data_path=gs://cloudfire_lyrical-edition-273206/fire_dataset/tfrecords-dataset-7/ \
    --eval_data_path=gs://cloudfire_lyrical-edition-273206/fire_dataset/tfrecords-dataset-eval/

gs://cloudfire_lyrical-edition-273206/model_dir77 us-central1 FireSage_200605_205257
jobId: FireSage_200605_205257
state: QUEUED


Job [FireSage_200605_205257] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe FireSage_200605_205257

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs FireSage_200605_205257


In [9]:
%%bash
python --version

Python 3.7.6
